In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os
import time
import json

# Ustawienie urządzenia: GPU, jeśli jest dostępne, w przeciwnym razie CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 1. Przygotowanie niestandardowego datasetu do wczytywania danych tabelarycznych i obrazów
class CarDataset(Dataset):
    def __init__(self, json_file, image_dir, transform=None):
        self.data = pd.read_json(json_file, lines=True)
        self.image_dir = image_dir
        self.transform = transform

        # One-Hot Encoding dla zmiennych kategorycznych
        self.data = pd.get_dummies(self.data, columns=['fuel_type', 'gearbox', 'model', 'car_type_main'], drop_first=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Ładowanie obrazu
        img_name = os.path.join(self.image_dir, self.data.iloc[idx]['img_local'])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        # Pobranie danych tabelarycznych (bez kolumny price i img_local)
        tabular_data = self.data.drop(columns=['price', 'img_local']).iloc[idx].values.astype(float)
        
        # Pobranie ceny jako celu
        price = self.data.iloc[idx]['price']
        
        return {
            'image': image,
            'tabular': torch.tensor(tabular_data, dtype=torch.float32),
            'price': torch.tensor(price, dtype=torch.float32)
        }

# 2. Definicja modelu hybrydowego: CNN + MLP
class HybridCarPricePredictor(nn.Module):
    def __init__(self, tabular_input_dim):
        super(HybridCarPricePredictor, self).__init__()
        
        # Ekstraktor cech obrazu - ResNet (bez warstwy końcowej klasyfikacji)
        self.cnn = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # Usuwamy ostatnią warstwę FC
        self.cnn_out_features = 512  # Wyjście ResNet18 (512 cech)

        # Sieć MLP dla danych tabelarycznych
        self.tabular_mlp = nn.Sequential(
            nn.Linear(tabular_input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )

        # Połączenie wyjść CNN i MLP
        self.fc = nn.Sequential(
            nn.Linear(self.cnn_out_features + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # Wyjście przewidujące cenę
        )

    def forward(self, image, tabular):
        # Przepuszczenie obrazu przez CNN
        cnn_features = self.cnn(image)
        cnn_features = cnn_features.view(cnn_features.size(0), -1)

        # Przepuszczenie danych tabelarycznych przez MLP
        tabular_features = self.tabular_mlp(tabular)

        # Połączenie cech z CNN i MLP
        combined_features = torch.cat((cnn_features, tabular_features), dim=1)

        # Predykcja ceny
        output = self.fc(combined_features)
        return output

# 3. Przygotowanie transformacji obrazu
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 4. Funkcja ewaluacji modelu
def evaluate_model(model, dataloader):
    model.eval()  # Ustawienie modelu w tryb ewaluacji
    total_loss = 0.0
    criterion = nn.MSELoss()
    
    with torch.no_grad():  # Wyłączenie gradientów dla oszczędności pamięci
        for batch in dataloader:
            images = batch['image'].to(device)
            tabular_data = batch['tabular'].to(device)
            prices = batch['price'].to(device)
            
            outputs = model(images, tabular_data).squeeze()
            loss = criterion(outputs, prices)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    print(f"Test Loss (MSE): {avg_loss:.4f}")
    return avg_loss

# 5. Przygotowanie danych i DataLoadera
image_dir = 'data_img'
train_json = 'train_data_with_car_type.json'
test_json = 'test_data_with_car_type.json'

train_dataset = CarDataset(json_file=train_json, image_dir=image_dir, transform=transform)
test_dataset = CarDataset(json_file=test_json, image_dir=image_dir, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 6. Inicjalizacja modelu, optymalizatora i funkcji straty
tabular_input_dim = train_dataset[0]['tabular'].shape[0]
model = HybridCarPricePredictor(tabular_input_dim).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 7. Trenowanie modelu
num_epochs = 30
for epoch in range(num_epochs):
    start_time = time.time()  # Start pomiaru czasu dla epoki

    model.train()
    running_loss = 0.0
    for batch in train_dataloader:
        images = batch['image'].to(device)
        tabular_data = batch['tabular'].to(device)
        prices = batch['price'].to(device)

        optimizer.zero_grad()

        # Forward pass i obliczanie straty
        outputs = model(images, tabular_data).squeeze()
        loss = criterion(outputs, prices)

        # Backward pass i optymalizacja
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Obliczenie średniego błędu i czasu dla epoki
    avg_loss = running_loss / len(train_dataloader)
    epoch_duration = time.time() - start_time
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Duration: {epoch_duration:.2f} seconds")

# 8. Zapis wytrenowanego modelu
torch.save(model.state_dict(), 'hybrid_car_price_predictor.pth')




In [ ]:
model.load_state_dict(torch.load('hybrid_car_price_predictor.pth'))
evaluate_model(model, test_dataloader)